In [1]:
import requests
import time
import re
import pandas as pd
import numpy as np
from tqdm import tqdm
from bs4 import BeautifulSoup
from datetime import date

In [2]:
GLOBAL = 'https://www.gipernn.ru'
LINK = 'https://www.gipernn.ru/arenda-domov/nizhegorodskaya-oblast'

Добыча колличества ссылок

In [3]:
html = requests.get(LINK)
soup = BeautifulSoup(html.text, 'html.parser')
count = soup.find('div', class_='count')

count = int(count.text.split(':')[1].strip())
page_count = int(np.ceil(count / 30))

print(count)
print(page_count)

40
2


Составляем массив ссылок

In [4]:
links = []
for page in tqdm(range(page_count + 1), ascii=True):
    html = requests.get(f"{LINK}?page={page}")
    soup = BeautifulSoup(html.text, 'html.parser')
    trs = soup.find('tbody').find_all('tr')
    for tr in trs:
        link = tr.find('a')
        if link:
            link = link.get('href')
            link = GLOBAL + link
            links.append(link)
    links = list(set(links))
print(len(links))   


100%|##########| 3/3 [00:00<00:00,  3.31it/s]

40


Составление своей базы данных

In [5]:
df = pd.DataFrame()
for link in tqdm(links, ascii=True):
    html = requests.get(link)
    soup = BeautifulSoup(html.text, 'html.parser')

    tmp = {} # словарь для временных данных
    tmp['Ссылка'] = link

    header = soup.find('h1')
    if header:
        header = header.text.strip().replace('\xa0', ' ')
    tmp['Заголовок'] = header

    price = soup.find('div', class_ = 'price')
    if price:
        price = price.text.replace('\xa0', ' ').strip().split('  ')
    tmp['Цена'] = price

    description = soup.find('div', class_='formatted-text')
    if description:
        description = description.text.strip().replace('\xa0', ' ')
    tmp['Описание'] = description
    
    try:
        w0 = soup.find('table', id = 'w0')
        if w0:
            w0 = w0.find_all('tr')
            for tr in w0:
                key = tr.find('th').text.strip()
                val = tr.find('td').text.strip()
                tmp[key] = val
    except:
        print('w0 not found')
 
    try:
        w2 = soup.find('table', id = 'w2')
        if w2:
            w2 = w2.find_all('tr')
            for tr in w2:
                key = tr.find('th').text.strip()
                val = tr.find('a').text.strip()
                if key not in tmp.keys(): # записи в 2х таблицах могут 
                    tmp[key] = val # пересекаться, а зачем нам копии?
    except:
        print('w2 not found')

    try:
        properties = soup.find('div', class_ = 'properties m-t-2')
        if properties:
            properties = properties.find_all('div', class_ = 'property')
            for property in properties:
                key = property.find('div', class_ = 'property-name').text.strip()
                val = property.find('div', class_ = 'property-value').text.strip()
                if key not in tmp.keys(): # записи в 2х таблицах могут 
                    tmp[key] = val # пересекаться, а зачем нам копии?
    except:
        print('properties m-t-2 not found')
    
    try:
        buildings = soup.find('div', 'around').find_all('div', 'group')
        for building in buildings:
            name = building.find('div','group-body').find('span').text
            dist = building.find('div','group-body').find('div', 'company').find('span', 'company__distance').text
            tmp[name] = dist   
    except:
        pass

    df = pd.concat([df, pd.DataFrame(tmp)], ignore_index=True)

df['Источник'] = 'gipernn'
df['Сегмент'] = 'Жилая недвижимость'
df['Подсегмент'] = 'Аренда домов/квартир/таунхаусов'
df['Город'] = 'Нижний Новгород'
df['Дата парсинга'] = str(date.today())

100%|##########| 40/40 [00:09<00:00,  4.12it/s]


Посмотрим на результаты вкрадце

In [6]:
df.head()

,Ссылка,Заголовок,Цена,Описание,№,Район,Адрес,Объект,Площадь,Площадь участка,...,Гараж,Туалет,Торг уместен,Бассейн,Телефон,Источник,Сегмент,Подсегмент,Город,Дата парсинга
0,https://www.gipernn.ru/arenda-domov/dom-poselo...,"Аренда дома на посёлок Дубёнки, ул. ГорнаяПрио...",14 000 руб./мес.,"Сдаю дом в Приокском районе, на длительный сро...",2411828,Приокский,"посёлок Дубёнки, ул. Горная",дом,62 / 32 / 13,5 соток,...,есть,в доме,NaN,NaN,NaN,gipernn,Жилая недвижимость,Аренда домов/квартир/таунхаусов,Нижний Новгород,2022-12-21
1,https://www.gipernn.ru/arenda-domov/dom-ul-zel...,Аренда дома на ул. ЗеленодольскаяКанавинский р...,13 000 руб./мес.,Сдаю частный дом в Канавинском районе. Дом нах...,2766431,Канавинский,ул. Зеленодольская,дом,65 / 45 / 10,5 соток,...,NaN,в доме,NaN,NaN,NaN,gipernn,Жилая недвижимость,Аренда домов/квартир/таунхаусов,Нижний Новгород,2022-12-21
2,https://www.gipernn.ru/arenda-domov/taunhaus-g...,"Аренда таунхауса (город Богородск, улица Турко...",35 000 руб./мес.,"Сдам дом, можно с последующим выкупом. Новый д...",2844418,Богородский муниципальный,"город Богородск, улица Туркова",таунхаус,300 / 100 / 30,10 соток,...,есть,в доме,NaN,NaN,NaN,gipernn,Жилая недвижимость,Аренда домов/квартир/таунхаусов,Нижний Новгород,2022-12-21
3,https://www.gipernn.ru/arenda-domov/dom-gorod-...,"Аренда дома (город Арзамас, Светлая улица)горо...",25 000 руб./мес.,Сдаётся дом порядочным платёжеспособным арен...,2837287,городской округ Арзамас,"город Арзамас, Светлая улица",дом,200 / 150 / 15,8 соток,...,NaN,NaN,NaN,NaN,NaN,gipernn,Жилая недвижимость,Аренда домов/квартир/таунхаусов,Нижний Новгород,2022-12-21
4,https://www.gipernn.ru/arenda-domov/dom-derevn...,Аренда дома (деревня Надеждино)Воротынский гор...,14 000 руб./мес.,"Сдам дом 70 кв.м. в д.Надеждино, Воротынский р...",2779432,Воротынский городской округ,деревня Надеждино,дом,70 / 40 / 15,40 соток,...,NaN,в доме,NaN,NaN,NaN,gipernn,Жилая недвижимость,Аренда домов/квартир/таунхаусов,Нижний Новгород,2022-12-21


Экспортируем таблицу

In [7]:
df.to_excel('cottage-rent.xlsx', index=False)